# Fine Tune BERT for sentiment analysis

This file fine-tunes `bert-base-uncased` on `argilla/twitter-coronavirus`. The dataset can be found at [here](https://huggingface.co/datasets/argilla/twitter-coronavirus)

This model is used for generating sentiment scores.

In [ ]:
!pip install datasets transformers accelerate evaluate

In [ ]:
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import evaluate
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Load Data

In [ ]:
data = load_dataset("argilla/twitter-coronavirus")
data = data.remove_columns(['inputs', 'prediction_agent', 'annotation', 'annotation_agent', 'multi_label', 'explanation', 'id', 'metadata', 'status', 'event_timestamp', 'metrics'])
data = data["train"].train_test_split(test_size=0.3)
data['train'] = data['train'].train_test_split(test_size=0.3)
data['dev'] = data['train']['test']
data['train'] = data['train']['train']
data

DatasetDict({
    train: Dataset({
        features: ['text', 'prediction'],
        num_rows: 22027
    })
    test: Dataset({
        features: ['text', 'prediction'],
        num_rows: 13487
    })
    dev: Dataset({
        features: ['text', 'prediction'],
        num_rows: 9441
    })
})

## Pre-processing

In [ ]:
label2idx = {'Extremely Negative': 0, 'Negative': 1, 'Neutral':2, 'Positive':3, 'Extremely Positive':4}
idx2label = {0: 'Extremely Negative', 1: 'Negative', 2: 'Neutral', 3: 'Positive', 4: 'Extremely Positive'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def preprocessing_func(examples):
    inputs = tokenizer(examples['text'], truncation=True)
    inputs['label'] = [label2idx[each[0]['label']] for each in examples['prediction']]
    return inputs

tokenized_data = data.map(preprocessing_func, batched=True, remove_columns=data['test'].column_names)
tokenized_data

Map:   0%|          | 0/22027 [00:00<?, ? examples/s]

Map:   0%|          | 0/13487 [00:00<?, ? examples/s]

Map:   0%|          | 0/9441 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 22027
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 13487
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 9441
    })
})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

## Training

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    matrix = accuracy.compute(predictions=predictions, references=labels)
    matrix['f1'] = f1_score(labels, predictions, average='weighted')
    return matrix

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=5, id2label=idx2label, label2id=label2idx
)

training_args = TrainingArguments(
    output_dir="model-output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model('bert-base-sentiment-analysis')